<a href="https://colab.research.google.com/github/vanrocha/drone-de-resgate/blob/main/Drone_de_Resgate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dronekit import connect, VehicleMode, LocationGlobalRelative, Command # Importa funcionalidades da biblioteca DroneKit
import time # Biblioteca para lidar com tempo
import math # Biblioteca para operações matemáticas
import pygame # Biblioteca para lidar com áudio
from flask import Flask, request, jsonify, send_from_directory # Importa funcionalidades da biblioteca Flask
from functools import wraps # Importa funcionalidades para wraps
from werkzeug.utils import secure_filename # Importa funcionalidades para lidar com segurança de nomes de arquivos
import os # Biblioteca para interagir com o sistema operacional
import jwt # Biblioteca para lidar com tokens JWT
import datetime # Biblioteca para lidar com datas e horas
from dronekit import mavutil # Importa funcionalidades da biblioteca mavutil
from google.cloud import aiplatform # Importa a biblioteca do AI Platform

# Cria a aplicação Flask
app = Flask(__name__)
# Configura o diretório para armazenar os arquivos de áudio
app.config['UPLOAD_FOLDER'] = 'mensagens'
# Define as extensões de arquivo permitidas para upload
app.config['ALLOWED_EXTENSIONS'] = {'mp3', 'jpg', 'jpeg', 'png'}
# Define a chave secreta para assinar os tokens JWT (**MANTENHA ISSO SEGURO**)
SECRET_KEY = 'sua_chave_secreta_aqui'

# Conecta-se ao drone (substitua 'COM*' pela porta serial correta)
vehicle = connect('COM*', wait_ready=True)

# --- Funções Auxiliares ---

# Função para verificar se a extensão do arquivo é permitida
def allowed_file(filename):
  return '.' in filename and \
     filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

# Função para gerar um token JWT para autenticação
def gerar_token(username):
  # Define o payload do token com o nome de usuário e tempo de expiração
  payload = {
    'username': username,
    'exp': datetime.datetime.utcnow() + datetime.timedelta(minutes=30)
  }
  # Gera o token JWT usando a chave secreta e o algoritmo HS256
  token = jwt.encode(payload, SECRET_KEY, algorithm='HS256')
  return token

# Decorador para exigir autenticação por token JWT
def token_obrigatorio(f):
  @wraps(f)
  def decorated(*args, **kwargs):
    # Obtém o token do cabeçalho Authorization
    token = request.headers.get('Authorization')
    if not token:
      # Retorna erro 401 se o token estiver ausente
      return jsonify({'message': 'Token ausente'}), 401
    try:
      # Decodifica o token usando a chave secreta
      data = jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
    except:
      # Retorna erro 401 se o token for inválido
      return jsonify({'message': 'Token inválido'}), 401
    # Se o token for válido, chama a função original
    return f(*args, **kwargs)
  return decorated

# --- Funções da API ---

# Função para reproduzir mensagem de áudio
def falar(mensagem):
  # Inicializa o mixer do Pygame
  pygame.mixer.init()
  # Carrega o arquivo de áudio da pasta 'mensagens'
  pygame.mixer.music.load(os.path.join(app.config['UPLOAD_FOLDER'], f'{mensagem}.mp3'))
  # Inicia a reprodução da mensagem
  pygame.mixer.music.play()
  # Aguarda a reprodução da mensagem terminar
  while pygame.mixer.music.get_busy():
    time.sleep(1)

# Função para enviar o drone a um local (latitude, longitude, altitude)
def ir_para(latitude, longitude, altitude):
  # Define o ponto de destino usando LocationGlobalRelative
  wp = LocationGlobalRelative(latitude, longitude, altitude)
  # Define o modo de voo como guiado e envia o drone ao ponto
  vehicle.simple_goto(wp)
  # Aguarda o drone atingir o ponto de destino
  while vehicle.mode.name=="GUIDED":
    # Calcula a distância até o ponto de destino
    distancia = get_distance_metres(vehicle.location.global_relative_frame, wp)
    print(f"Distância até o ponto: {distancia:.2f} metros")
    # Se a distância for menor ou igual a 1 metro, considera o ponto atingido
    if distancia<=1:
      print("Ponto atingido!")
      break
    time.sleep(1)

# Função para realizar busca em espiral
def busca_espiral(centro_lat, centro_lon, altitude, raio_inicial, raio_final, espaçamento):
  # Define o modo de voo como guiado
  vehicle.mode = VehicleMode("GUIDED")
  # Obtém a lista de comandos do drone
  cmds = vehicle.commands
  # Limpa a lista de comandos
  cmds.clear()
  # Define o ponto inicial da espiral usando LocationGlobalRelative
  proximo_ponto = LocationGlobalRelative(centro_lat, centro_lon, altitude)
  # Define o raio inicial da espiral
  raio = raio_inicial
  # Define o ângulo inicial da espiral
  angulo = 0
  # Loop para gerar os pontos da espiral
  while raio <= raio_final:
      # Calcula as coordenadas do próximo ponto da espiral
      proximo_ponto.lat = centro_lat + raio * math.cos(angulo)
      proximo_ponto.lon = centro_lon + raio * math.sin(angulo)
      # Adiciona o ponto à lista de comandos usando Command
      cmds.add(Command(0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT,
                      mavutil.mavlink.MAV_CMD_NAV_WAYPOINT, 0, 0, 0, 0, 0, 0,
                      proximo_ponto.lat, proximo_ponto.lon, proximo_ponto.alt))
      # Incrementa o ângulo da espiral
      angulo += espaçamento / raio
      # Incrementa o raio da espiral
      raio += espaçamento / (2 * math.pi)
  # Envia os comandos para o drone
  cmds.upload()
  # Define o modo de voo como automático para iniciar a busca em espiral
  vehicle.mode = VehicleMode("AUTO")

# Simulação de gravação de vídeo (substitua pela lógica real da sua câmera)
gravacao_ativa = False

# Função para simular o início da gravação de vídeo
def iniciar_gravacao():
  global gravacao_ativa
  gravacao_ativa = True
  print("Gravação iniciada (simulado)")

# Função para simular o fim da gravação de vídeo
def parar_gravacao():
  global gravacao_ativa
  gravacao_ativa = False
  print("Gravação parada (simulado)")

# --- Funções para reconhecimento facial com Google AI Platform ---

# Credenciais do Google Cloud (configure as suas)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "caminho/para/seu/arquivo/de/credenciais.json"
# Nome do projeto e endpoint do modelo no AI Platform
PROJECT_ID = "seu-projeto-id"
ENDPOINT_ID = "seu-endpoint-id"

# Função para detecção de pessoas usando o AI Platform
def detectar_pessoas(imagem):
  # Inicializa o AI Platform
  aiplatform.init(project=PROJECT_ID, location="us-central1")
  # Obtém o endpoint do modelo no AI Platform
  endpoint = aiplatform.Endpoint(ENDPOINT_ID)
  # Envia a imagem para o modelo e obtém a resposta
  response = endpoint.predict(instances=[{"content": imagem}])
  # Retorna as predições do modelo
  return response.predictions

# --- Endpoints da API ---

# Endpoint para login e obtenção de token JWT
@app.route('/login', methods=['POST'])
def login():
  # Obtém as credenciais do corpo da requisição
  data = request.get_json()
  username = data.get('username')
  password = data.get('password')
  # Verifica as credenciais (substitua por sua lógica real de autenticação)
  if username == 'admin' and password == 'senha_secreta':
    # Gera o token JWT
    token = gerar_token(username)
    # Retorna o token JWT na resposta
    return jsonify({'token': token}), 200
  else:
    return jsonify({'message': 'Credenciais inválidas'}), 401

# Endpoint para mover o drone para um local específico (requer autenticação)
@app.route('/ir', methods=['POST'])
@token_obrigatorio
def ir_endpoint():
  # Obtém os dados da requisição
  data = request.get_json()
  latitude = data.get('latitude')
  longitude = data.get('longitude')
  altitude = data.get('altitude')
  # Verifica se os parâmetros foram fornecidos
  if latitude is None or longitude is None or altitude is None:
    return jsonify({'error': 'Latitude, longitude e altitude são obrigatórios'}), 400
  # Chama a função para mover o drone
  ir_para(latitude, longitude, altitude)
  return jsonify({'message': 'Drone indo para o local'}), 200

# Endpoint para reproduzir mensagem de áudio (requer autenticação)
@app.route('/falar', methods=['POST'])
@token_obrigatorio
def falar_endpoint():
  # Obtém os dados da requisição
  data = request.get_json()
  mensagem = data.get('mensagem')
  # Verifica se o nome da mensagem foi fornecido
  if mensagem is None:
    return jsonify({'error': 'Nome da mensagem é obrigatório'}), 400
  # Chama a função para reproduzir a mensagem
  falar(mensagem)
  return jsonify({'message': 'Mensagem reproduzida'}), 200

# Endpoint para fazer upload de arquivos de áudio (requer autenticação)
@app.route('/upload', methods=['POST'])
@token_obrigatorio
def upload_file():
  # Verifica se o arquivo foi enviado na requisição
  if 'file' not in request.files:
    return jsonify({'error': 'Nenhum arquivo encontrado'}), 400
  # Obtém o arquivo da requisição
  file = request.files['file']
  # Verifica se o nome do arquivo está presente
  if file.filename == '':
    return jsonify({'error': 'Nome de arquivo vazio'}), 400
  # Verifica se o arquivo é permitido
  if file and allowed_file(file.filename):
    # Obtém um nome seguro para o arquivo
    filename = secure_filename(file.filename)
    # Salva o arquivo no servidor
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    return jsonify({'message': f'Arquivo {filename} enviado com sucesso'}), 201
  else:
    return jsonify({'error': 'Extensão de arquivo não permitida'}), 400

# Endpoint para servir um arquivo de áudio (requer autenticação)
@app.route('/mensagens/<filename>')
@token_obrigatorio
def servir_audio(filename):
  return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# Endpoint para iniciar a gravação (requer autenticação)
@app.route('/gravar/iniciar', methods=['POST'])
@token_obrigatorio
def iniciar_gravacao_endpoint():
  iniciar_gravacao()
  return jsonify({'message': 'Gravação iniciada'}), 200

# Endpoint para parar a gravação (requer autenticação)
@app.route('/gravar/parar', methods=['POST'])
@token_obrigatorio
def parar_gravacao_endpoint():
  parar_gravacao()
  return jsonify({'message': 'Gravação parada'}), 200

# Endpoint para detecção de pessoas (requer autenticação)
@app.route('/detectar_pessoas', methods=['POST'])
@token_obrigatorio
def detectar_pessoas_endpoint():
  # Verifica se o arquivo de imagem foi enviado na requisição
  if 'file' not in request.files:
    return jsonify({'error': 'Nenhum arquivo encontrado'}), 400
  # Obtém o arquivo de imagem da requisição
  file = request.files['file']
  # Verifica se o nome do arquivo de imagem está presente
  if file.filename == '':
    return jsonify({'error': 'Nome de arquivo vazio'}), 400
  # Carrega a imagem em memória (ajuste de acordo com o formato)
  imagem = file.read()
  # Chama a função de detecção de pessoas
  resultados = detectar_pessoas(imagem)
  # Processa a resposta do modelo (adapte de acordo com a estrutura do seu modelo)
  coordenadas = []
  for resultado in resultados:
      for deteccao in resultado:
          # Verifica se a detecção é uma pessoa e se a confiança é maior que 0.5
          if deteccao['displayName'] == 'pessoa' and deteccao['confidenceScore'] > 0.5:
              # Adiciona as coordenadas da caixa delimitadora à lista
              coordenadas.append(deteccao['boundingBox'])
  # Retorna as coordenadas das pessoas encontradas em formato JSON
  return jsonify({'pessoas': coordenadas}), 200

# Inicia a aplicação Flask em modo de depuração
if __name__ == '__main__':
  app.run(debug=True)

Obs:

- Configure o ambiente, substituindo os placeholders pelas suas informações reais.
- Ajuste a lógica da detecção facial de acordo com o modelo no Google AI Platform.
- Explore a biblioteca DroneKit e a documentação do drone para implementar outras funcionalidades e aprimorar a lógica de voo!